In [ ]:
# MTF073 Computational Fluid Dynamics
# Task 1: 2D diffusion
# Håkan Nilsson, 2023
# Department of Mechanics and Maritime Sciences
# Division of Fluid Dynamics
# Note that this is not efficient code. It is for educational purposes!

# Clear all variables when running entire code:
from IPython import get_ipython
get_ipython().run_line_magic('reset', '-sf')
# Packages needed
import numpy as np
import matplotlib.pyplot as plt
# Close all plots when running entire code:
plt.close('all')
# Set default font size in plots:
plt.rcParams.update({'font.size': 12})
import os # For saving plots

#===================== Inputs =====================

# Geometric and mesh inputs

L = 1 # Length of the domain in X direction
H = 2 # Length of the domain in Y direction
mI = 8 # Number of mesh points X direction.
mJ = 8 # Number of mesh points Y direction.
mesh_type = 'equidistant' # Set 'equidistant' or 'non-equidistant'

# Solver inputs

nIter  =  1000 # set maximum number of iterations
resTol =  0.001 # set convergence criteria for residuals

#====================== Code ======================

# Preparation of "nan", to fill empty slots in consistently numbered arrays.
# This makes it easier to check in Variable Explorer that values that should
# never be set are never set (or used). Plots simply omit nan values.
nan = float("nan")

# Allocate arrays (nan used to make clear where values need to be set)
# Note that some arrays could actually be 1D since they only have a variation
# in one direction, but they are kept 2D so the indexing is similar for all.
nI = mI + 1                    # Number of nodes in X direction, incl. boundaries
nJ = mJ + 1                    # Number of nodes in Y direction, incl. boundaries
pointX = np.zeros((mI,mJ))*nan # X coords of the mesh points
pointY = np.zeros((mI,mJ))*nan # Y coords of the mesh points
nodeX  = np.zeros((nI,nJ))*nan # X coords of the nodes
nodeY  = np.zeros((nI,nJ))*nan # Y coords of the nodes
dx_PE  = np.zeros((nI,nJ))*nan # X distance to east node
dx_WP  = np.zeros((nI,nJ))*nan # X distance to west node
dy_PN  = np.zeros((nI,nJ))*nan # Y distance to north node
dy_SP  = np.zeros((nI,nJ))*nan # Y distance to south node
dx_we  = np.zeros((nI,nJ))*nan # X size of the control volume
dy_sn  = np.zeros((nI,nJ))*nan # Y size of the control volume
aE     = np.zeros((nI,nJ))*nan # Array for east coefficient, in nodes
aW     = np.zeros((nI,nJ))*nan # Array for wect coefficient, in nodes
aN     = np.zeros((nI,nJ))*nan # Array for north coefficient, in nodes
aS     = np.zeros((nI,nJ))*nan # Array for south coefficient, in nodes
aP     = np.zeros((nI,nJ))*nan # Array for central coefficient, in nodes
Su     = np.zeros((nI,nJ))*nan # Array for source term for temperature, in nodes
Sp     = np.zeros((nI,nJ))*nan # Array for source term for temperature, in nodes
T      = np.zeros((nI,nJ))*nan # Array for temperature, in nodes
k      = np.zeros((nI,nJ))*nan # Array for conductivity, in nodes
k_e    = np.zeros((nI,nJ))*nan # Array for conductivity at east face
k_w    = np.zeros((nI,nJ))*nan # Array for conductivity at west face
k_n    = np.zeros((nI,nJ))*nan # Array for conductivity at north face
k_s    = np.zeros((nI,nJ))*nan # Array for conductivity at south face
res    = []                    # Array for appending residual each iteration



In [ ]:
# Generate mesh and compute geometric variables

if mesh_type == 'equidistant':
    # Calculate mesh point coordinates:
    for i in range(0, mI):
        for j in range(0, mJ):
            pointX[i,j] = i*L/(mI - 1)
            pointY[i,j] = j*H/(mJ - 1)
            

# Smaller to the top and bottom
#   -------------------------
#   | | | | | | | | | | | | |
#   -------------------------
#   | | |   |   |   |   | | |
#   | | |   |   |   |   | | |
#   -------------------------
#   | | | | | | | | | | | | |
#   -------------------------

y1 = np.linspace(0,H/4,(mJ)//2)
y2 = np.linspace(H/8,3*H/4,(mJ)//2)
y3 = np.linspace(3*H/4,H,(mJ)//2)
y = np.concatenate((y1,y2[1:],y3[1:]))

x1 = np.linspace(0,L/4,(mI)//2)
x2 = np.linspace(L/4,3*L/4,(mI)//2)
x3 = np.linspace(3*L/4,L,(mI)//2)
x4 = np.concatenate((x1,x2,x3))
x = np.concatenate((x1,x2[1:],x3[1:]))

print(x4)
print(x)
if mesh_type == 'non-equidistant':
    # Calculate mesh point coordinates with stretching

    for i in range(0, mI):
        for j in range(0, mJ):
            
            pointX[i, j] = x[i]
            pointY[i, j] = y[j]

   
            
   

# Calculate node coordinates (same for equidistant and non-equidistant):
# Internal nodes:
for i in range(0, nI):
    for j in range(0, nJ):
        if i > 0 and i < nI-1:
            nodeX[i,j] = 0.5*(pointX[i,0] + pointX[i-1,0])
        if j > 0 and j < nJ-1:
            nodeY[i,j] = 0.5*(pointY[0,j] + pointY[0,j-1])
# Boundary nodes:
nodeX[0,:] = 0   # Note: corner points needed for contour plot
nodeY[:,0] = 0   # Note: corner points needed for contour plot
nodeX[-1,:] = L # Note: corner points needed for contour plot
nodeY[:,-1] = H # Note: corner points needed for contour plot


In [ ]:
# Plot mesh
plt.figure()
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('Computational mesh')
plt.axis('equal')
plt.vlines(pointX[:,0],0,H,colors = 'k',linestyles = 'dashed')
plt.hlines(pointY[0,:],0,L,colors = 'k',linestyles = 'dashed')
plt.plot(nodeX, nodeY, 'ro')
plt.tight_layout()
plt.show()



In [ ]:
# Calculate distances
# Keep 'nan' where values are not needed!
for i in range(1, nI-1):
    for j in range(1, nJ-1):
        dx_PE[i,j] = nodeX[i+1,j] - nodeX[i,j]
        dx_WP[i,j] = nodeX[i,j] - nodeX[i-1,j]
        dy_PN[i,j] = nodeY[i,j+1] - nodeY[i,j]
        dy_SP[i,j] = nodeY[i,j] - nodeY[i,j-1]
        dx_we[i,j] = pointX[i,j] - pointX[i-1,j]
        dy_sn[i,j] = pointY[i,j] - pointY[i,j-1]


# Copy the boundary distances
for j in range(0, nJ):
    i = -1
    dx_we[i,j] = dx_we[i-1,j]
    i = 0
    dx_we[i,j] = dx_we[i+1,j]

for i in range(0, nI):
    j = -1
    dy_sn[i,j] = dy_sn[i,j-1]
    j = 0
    dy_sn[i,j] = dy_sn[i,j+1]

print(dx_we[1:-1,1:-1])


In [ ]:
# Initialize dependent variable array and Dirichlet boundary conditions
# Note that a value is needed in all nodes for contour plot
# ADD CODE HERE

T = np.zeros_like(T)
      
T[:,0] = 10
T[-1,:] = 20
for i in range(0,nI):
    T[i,-1] = 5 + 3*(1 + 5*nodeX[i,j]/L)

# g_y = np.linspace(T[i,0], T[i,-1],nJ) # linear T on the given x coordinate from the bottom to the top
# g_x = np.linspace(T[0,j], T[-1,j],nI) # linear T on the given y coordinate from the bottom to the top
# for i in range(0,nI-2):
#     for j in range(1,nJ-2):
#         T[i,j] = (g_x[i]+g_y[j])/2


# Plot temperature contour
plt.figure()
plt.title('Temperature distribution')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')
tempmap=plt.contourf(nodeX.T,nodeY.T,T.T,cmap='coolwarm',levels=30)
cbar=plt.colorbar(tempmap)
cbar.set_label('Temperature [K]')
plt.tight_layout()


In [ ]:

c1 = 25
c2 = 0.25
T1 = 10
for iteration in range(nIter):
    
    # Update conductivity arrays k, k_e, k_w, k_n, k_s, according to your case:
    # (could be moved to before iteration loop if independent of solution,
    # but keep here if you want to easily test different cases)
    # Keep 'nan' where values are not needed!
    # ADD CODE HERE.

    # Boundary k
    for i in range(0,nI):
        for j in range(0, nJ):
            k[i,j] = 16*(nodeY[i,j]/H + 30*T[i,j]/T1)




    # interpolate to get the surface values for k
    for i in range(1,nI-1):
        for j in range(1,nJ-1):
            k_e[i,j] = k[i,j] + (k[i+1,j] - k[i,j]) * dx_we[i,j]/(dx_we[i,j] + dx_we[i+1,j])
            k_w[i,j] = k[i,j] + (k[i-1,j] - k[i,j]) * dx_we[i,j]/(dx_we[i,j] + dx_we[i-1,j])
            k_n[i,j] = k[i,j] + (k[i,j+1] - k[i,j]) * dy_sn[i,j]/(dy_sn[i,j] + dy_sn[i,j+1])
            k_s[i,j] = k[i,j] + (k[i,j-1] - k[i,j]) * dy_sn[i,j]/(dy_sn[i,j] + dy_sn[i,j-1])
    
     

    # Update source term array according to your case:
    # (could be moved to before iteration loop if independent of solution,
    # but keep here if you want to easily test different cases)
    # Keep 'nan' where values are not needed!
    # ADD CODE HERE.
    for i in range(0,nI-1):
        for j in range(0,nJ-1):
            Sp[i,j] = -15*c2*T[i,j] * dy_sn[i,j] * dx_we[i,j]
            Su[i,j] = 15*c1 * dy_sn[i,j] * dx_we[i,j]
    
    # Calculate coefficients:
    # (could be moved to before iteration loop if independent of solution)
    # Keep 'nan' where values are not needed!
    # Inner node neighbour coefficients:
    # (not caring about special treatment at boundaries):
    for i in range(1,nI-1):
        for j in range(1,nJ-1):
            Ae = dy_sn[i,j]
            Aw = dy_sn[i,j]
            As = dx_we[i,j]
            An = dx_we[i,j]

            aE[i,j] = Ae*k_e[i,j]/(dx_PE[i,j])
            aW[i,j] = Aw*k_w[i,j]/(dx_WP[i,j])
            aN[i,j] = An*k_n[i,j]/(dy_PN[i,j])
            aS[i,j] = As*k_s[i,j]/(dy_SP[i,j])
   
    # Modifications of aE and aW inside east and west boundaries:
    for j in range(1,nJ-1):
        i = nI-2 #East
        # aE[i,j] = 0

        i=1 #West
        aW[i,j] = 0


    # Modifications of aN and aS inside north and south boundaries:
    for i in range(1,nI-1):
        j = nJ-2 # North
        # aN[i,j] = 0

        j=1 # South
        # aS[i,j] = 0


    # Inner node central coefficients:
    for i in range(1,nI-1):
        for j in range(1,nJ-1):
            aP[i,j] = aS[i,j] + aN[i,j] + aE[i,j] + aW[i,j] - Sp[i,j]

    alpha = 1
    # Solve for T using Gauss-Seidel:
    for i in range(1, nI - 1):
        for j in range(1, nJ - 1):
            T[i, j] = ((aW[i, j] * T[i - 1, j] + aE[i, j] * T[i + 1, j] + aS[i, j] * T[i, j - 1] + aN[i, j] * T[i, j + 1] + Su[i, j]) / aP[i, j])

    
    # # Copy T to boundaries (and corners) where homegeneous Neumann is applied:
    # for j in range(1, nJ - 1):
    #     i = 0  # West boundary
    #     T[i, j] = T[i + 1, j]  # dT/dx = 0
    
    # Compute and print residuals (taking into account normalization):
    r = 0
    for i in range(1, nI-1):
        for j in range(1, nJ-1):
            r += abs(aP[i,j]*T[i,j] - (aW[i,j]*T[i-1,j] + aE[i,j]*T[i+1,j] + aN[i,j]*T[i,j+1] + aS[i,j]*T[i,j-1] + Su[i,j]))

    F = 0
    [dT_dx, dT_dy] = np.gradient(T, nodeX[:, 0], nodeY[0, :])

    # in y-direction
    for i in range(1,nI-1):
        # South boundary
        if dT_dy[i, 0] > 0:
            F += abs(k_s[i, 1] * dx_we[i, 1] * dT_dy[i, 1])
        # North boundary
        if dT_dy[i, -1] < 0:
            F += abs(k_n[i, -2] * dx_we[i, -2] * dT_dy[i, -2])
    # in x-direction
    for j in range(1,nJ-1):
        # West boundary
        if dT_dx[0, j] > 0:
            F += abs(k_w[1, j] * dy_sn[1, j] * dT_dx[1, j])
        # East boundary
        if dT_dx[-1, j] < 0:
            F += abs(k_e[-2, j] * dy_sn[-2, j] * dT_dx[-2, j])

    r /= F
    print('iteration: %5d, res = %.5e' % (iteration, r))
    
    # Append residual at present iteration to list of all residuals, for plotting:
    res.append(r)
    
    # Stop iterations if converged:
    if r < resTol:
        break

# Copy T to boundaries (and corners) where homegeneous Neumann is applied:
for j in range(1, nJ - 1):
    i = 0  # West boundary
    T[i, j] = T[i + 1, j]  # dT/dx = 0

# update to correct gradient after Neumann is applied
[dT_dx, dT_dy] = np.gradient(T, nodeX[:, 0], nodeY[0, :])



In [ ]:
# Plot temperature contour
plt.figure()
plt.title('Temperature distribution')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')
tempmap=plt.contourf(nodeX.T,nodeY.T,T.T,cmap='coolwarm',levels=30)
cbar=plt.colorbar(tempmap)
cbar.set_label('Temperature [K]')
plt.tight_layout()

if mesh_type == 'equidistant':
    plt.savefig('Figures/equidistant/temperatureDistribution.png')
if mesh_type == 'non-equidistant':
    plt.savefig('Figures/non_equidistant/temperatureDistribution.png')


In [ ]:
# Plot residual convergence
plt.figure()
plt.title('Residual convergence')
plt.xlabel('Iterations')
plt.ylabel('Residuals [-]')
resLength = np.arange(0,len(res),1)
plt.plot(resLength, res)
plt.grid()
plt.yscale('log')

if mesh_type == 'equidistant':
    plt.savefig('Figures/equidistant/residualconvergence.png')
if mesh_type == 'non-equidistant':
    plt.savefig('Figures/non_equidistant/residualconvergence.png')


In [ ]:
# Heat flux
plt.title('Temperature distribution')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')
tempmap=plt.contourf(nodeX.T,nodeY.T,T.T,cmap='coolwarm',levels=30)
cbar=plt.colorbar(tempmap)
cbar.set_label('Temperature [K]')
plt.tight_layout()
plt.quiver(nodeX[1:-1,1:-1], nodeY[1:-1,1:-1], -dT_dx[1:-1,1:-1], -dT_dy[1:-1,1:-1], width = 0.003)
plt.title('Heat flux')
if mesh_type == 'equidistant':
    plt.savefig('Figures/equidistant/heatflux.png')
if mesh_type == 'non-equidistant':
    plt.savefig('Figures/non_equidistant/heatflux.png')

In [ ]:
# Wall heat flux

# Calculate wall normal heat flux components at the boundary
# Calculate temperature gradient in x and y directions


boundary_nodes_top = np.logical_or.reduce(
    (nodeY == H)
) & ~(
    (nodeX == 0) & (nodeY == 0) |
    (nodeX == L) & (nodeY == 0) |
    (nodeX == 0) & (nodeY == H) |
    (nodeX == L) & (nodeY == H)
)
boundary_normal_heat_flux_top = -dT_dy[boundary_nodes_top]*k_n[boundary_nodes_top]

boundary_nodes_bottom = np.logical_or.reduce(
    (nodeY == 0)
) & ~(
    (nodeX == 0) & (nodeY == 0) |
    (nodeX == L) & (nodeY == 0) |
    (nodeX == 0) & (nodeY == H) |
    (nodeX == L) & (nodeY == H)
)
boundary_normal_heat_flux_bottom = -dT_dy[boundary_nodes_bottom]*k_s[boundary_nodes_bottom]



boundary_nodes_x = np.logical_or.reduce(
    ( nodeX == 0, nodeX == L)
) & ~(
    (nodeX == 0) & (nodeY == 0) |
    (nodeX == L) & (nodeY == 0) |
    (nodeX == 0) & (nodeY == H) |
    (nodeX == L) & (nodeY == H)
)

k_n[boundary_nodes_x] = 16*(nodeY[boundary_nodes_x]/H + 30*T[boundary_nodes_x]/T1)
boundary_normal_heat_flux_x = -dT_dx[boundary_nodes_x]*k_n[boundary_nodes_x]



# Create a quiver plot for wall normal heat flux at the boundary
plt.figure()
plt.title('Wall Normal Heat Flux')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')
plt.xlim(left = -1.5, right = L+1.5)

tempmap=plt.contourf(nodeX.T,nodeY.T,T.T,cmap='coolwarm',levels=30)
cbar=plt.colorbar(tempmap)

# Quiver plot for wall normal heat flux at the boundary
plt.quiver(nodeX[boundary_nodes_top], nodeY[boundary_nodes_top], np.zeros_like(boundary_normal_heat_flux_top), boundary_normal_heat_flux_top, width = 0.002)
plt.quiver(nodeX[boundary_nodes_bottom], nodeY[boundary_nodes_bottom], np.zeros_like(boundary_normal_heat_flux_bottom), boundary_normal_heat_flux_bottom, width = 0.002)
plt.quiver(nodeX[boundary_nodes_x], nodeY[boundary_nodes_x], boundary_normal_heat_flux_x, np.zeros_like(boundary_normal_heat_flux_x), width = 0.002)



plt.tight_layout()
if mesh_type == 'equidistant':
    plt.savefig('Figures/equidistant/wall_normal_heat_flux.png')
if mesh_type == 'non-equidistant':
    plt.savefig('Figures/non_equidistant/wall_normal_heat_flux.png')



In [ ]:
# import sympy as sp

# a_E,T_E, a_W,T_W, a_N,T_N, a_S,T_S ,C_1, C_2 , T_P= sp.symbols('a_E T_E  a_W T_W  a_N T_N  a_S T_S  C_1 C_2 T_P')
# eq1 = (a_E *T_E + a_W *T_W + a_N *T_N + a_S*T_S + 15*C_1) / (a_E + a_W + a_N + a_S - 15*C_2*T_P)/T_P == 1

# sp.solve(eq1, T_P)